# 互联网新闻情感分析

## 该模板也将作为标准模型模板

In [10]:
! ls /cos_person/ccf/data

Test_DataSet.csv	       pos_tfidf.json
Train_DataSet.csv	       sort_importance.csv
Train_DataSet_Label.csv        stopwords.txt
com_tfidf.json		       submit_example.csv
first_round_testing_data.csv   submit_example2.csv
first_round_training_data.csv  word_seg.txt
neg_tfidf.json


In [12]:
# 导入相关包
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import gc
import warnings
import json
import jieba
import re
import random
warnings.filterwarnings("ignore")
np.random.seed(2019)

In [13]:
path = "/cos_person/ccf/data/"
train = pd.read_csv(path+'Train_DataSet.csv')
test = pd.read_csv(path+'Test_DataSet.csv')
labels = pd.read_csv(path+'Train_DataSet_Label.csv') # 2 负 1 中 0 正
train = train.merge(labels, on=['id'], how='left')

In [14]:
train.label.value_counts()

1.0    3646
2.0    2931
0.0     763
Name: label, dtype: int64

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7345 entries, 0 to 7344
Data columns (total 4 columns):
id         7345 non-null object
title      7342 non-null object
content    7266 non-null object
label      7340 non-null float64
dtypes: float64(1), object(3)
memory usage: 286.9+ KB


In [27]:
# train[train["label"] == 0]

In [16]:
# 停用词
stopwordset = set()
with open(path+'stopwords.txt', 'r', encoding='utf8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
# stopwordset

In [ ]:
# def preprocess_text(content_lines,sentences,category):
#     for line in content_lines:
#         try:
#             segs=jieba.lcut(line)
#             segs = filter(lambda x:len(x)>1, segs)
#             segs = filter(lambda x:x not in stopwords, segs)
#             sentences.append((" ".join(segs), category))
#         except:
#             print(line)
#             continue

In [28]:
#分词模块
# 去标点符号，去停词，去英文，去数字

output = open(path+'word_seg.txt','w', encoding='utf8')

def seg_sent(art):
    r = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'  
    art = art.strip()
    art = re.sub(r, '', art)
    art = re.sub("[a-z]","",art)
    art = re.sub("[0-9]","",art)
    l = []
    seg_list = list(jieba.cut(art, cut_all = False))
    seg_list = filter(lambda x:len(x) > 1, seg_list) # 去除单 词
    for word in seg_list:
        if word not in stopwordset and word != ' ' and word != "\n" and word != "\n\n":
                output.write(word + ' ')
                l.append(word)
    return l

def seg_art_list(df):
    corpus = []
    for index, row in df.iterrows():
        title = row["title"]
        if title is np.nan:
            title = ""
        content = row["content"]
        if content is np.nan:
            content = ""
        text = title+" " + str(content)
        if not text:
            continue
        corpus.append((" ".join(seg_sent(text)), row["label"]))
    return corpus


pos_tfidf = seg_art_list(train[train.label == 0])
com_tfidf = seg_art_list(train[train.label == 1])
neg_tfidf = seg_art_list(train[train.label == 2])
# with open(path+"pos_tfidf.json", 'w', encoding='utf8') as fp:
#     json.dump(pos_tfidf, fp)
# with open(path+"com_tfidf.json", 'w', encoding='utf8') as fp:
#     json.dump(com_tfidf, fp)
# with open(path+"neg_tfidf.json", 'w', encoding='utf8') as fp:
#     json.dump(neg_tfidf, fp)

Building prefix dict from the default dictionary ...
08/29/2019 02:35:55 - DEBUG - jieba -   Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
08/29/2019 02:35:56 - DEBUG - jieba -   Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.012 seconds.
08/29/2019 02:35:56 - DEBUG - jieba -   Loading model cost 1.012 seconds.
Prefix dict has been built succesfully.
08/29/2019 02:35:56 - DEBUG - jieba -   Prefix dict has been built succesfully.


In [29]:
test['label'] = -1
test_tfidf = seg_art_list(test)

In [30]:
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics.scorer import make_scorer
from sklearn import linear_model
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score,accuracy_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GridSearchCV

In [199]:
# ?TfidfVectorizer

In [31]:
train_tfidf = pos_tfidf+com_tfidf+neg_tfidf
all_tfidf = train_tfidf+test_tfidf
X, y=zip(*all_tfidf)
# cv = CountVectorizer(ngram_range=(1,2), min_df=20,max_df = 0.9,lowercase=False)
# _X = cv.fit_transform(X).toarray()
# tfidf = TfidfVectorizer(ngram_range=(1,2))
tfidf = TfidfVectorizer()
_X = tfidf.fit_transform(X)
feature_names = list(tfidf.get_feature_names())
# 分层采样
y = np.array(y)[:len(train_tfidf)]
X = _X[:len(train_tfidf)]
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2019)
for train_index, test_index in split.split(X, y):
    X_train=X[train_index]
    y_train=y[train_index]
    X_test=X[test_index]
    y_test=y[test_index]

In [32]:
_X.shape

(14696, 238862)

In [35]:
X_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [245]:
# 训练
classifier= RandomForestClassifier(n_estimators=100,random_state=5,min_samples_leaf=2)
model = classifier.fit(X_train, y_train)
res = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
print(res)

y_pred = model.predict(X_test)
print("classifier result:")
print(clsr(y_test, y_pred, target_names=['pos', 'com', 'neg']))

print("confusion matrix")
print(metrics.confusion_matrix(y_test, y_pred))
# 评估
f1_score(y_test, y_pred, average='macro')

[0.71768707 0.72278912 0.73639456 0.7427598  0.70868825 0.72572402
 0.73083475 0.71843003 0.70819113 0.71331058]
classifier result:
              precision    recall  f1-score   support

         pos       0.25      0.01      0.01       153
         com       0.68      0.86      0.76       729
         neg       0.75      0.70      0.73       586

   micro avg       0.71      0.71      0.71      1468
   macro avg       0.56      0.52      0.50      1468
weighted avg       0.67      0.71      0.67      1468

confusion matrix
[[  1 115  37]
 [  3 629  97]
 [  0 176 410]]


0.5004297227518294

In [251]:
# 逻辑回归
grid_values = {'C':[1, 5, 10, 30]}  # 设定grid search的参数
model_LR = GridSearchCV(LR(penalty = 'l2', dual = True, random_state = 2019), grid_values, scoring = 'accuracy', cv = 10) 
model_LR.fit(X_train,y_train) # 数据灌进来
y_pred = model_LR.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6132281143133528

In [252]:
metrics.confusion_matrix(y_test, y_pred)

array([[ 34,  89,  30],
       [ 24, 612,  93],
       [  3, 157, 426]])

In [253]:
model_LR.best_params_
_model = model_LR.best_estimator_
y_pred = _model.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6132281143133528

In [241]:
grid_values = {'C':[10, 30]}  # 设定grid search的参数
model_LR = GridSearchCV(LR(penalty = 'l1', random_state = 2019), grid_values, scoring = 'accuracy', cv = 10) 
model_LR.fit(X_train,y_train) # 数据灌进来
y_pred = model_LR.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6028089828236213

In [234]:
# SVM
_model = SVC(kernel='linear')
_model.fit(X_train, y_train)
res = cross_val_score(_model, X_train, y_train, cv=10, scoring='accuracy')
print(res)
f1_score(y_test, _model.predict(X_test), average='macro')

[0.77380952 0.74319728 0.76190476 0.74787053 0.74787053 0.76149915
 0.75298126 0.76791809 0.7440273  0.73720137]


0.5861214870418736

In [250]:
# 贝叶斯
_model = MultinomialNB()
_model.fit(X_train, y_train)
res = cross_val_score(_model, X_train, y_train, cv=10, scoring='accuracy')
print(res)
f1_score(y_test, _model.predict(X_test), average='macro')
# accuracy_score(y_test, model.predict(X_test))

[0.72619048 0.72619048 0.72278912 0.72402044 0.7274276  0.70017036
 0.7120954  0.71501706 0.71501706 0.6894198 ]


0.48267669088308623

In [254]:
# 测试集
_test = _X[len(train_tfidf):]
y_pred = _model.predict(_test)
y_pred

array(['1', '2', '1', ..., '1', '1', '1'], dtype='<U2')

In [255]:
pd.DataFrame(y_pred)[0].value_counts()

1    4213
2    2838
0     305
Name: 0, dtype: int64

In [256]:
# 保存结果
result_pd = pd.DataFrame()
result_pd['id']=test['id']
result_pd['label'] = y_pred
result_pd.to_csv('menqi-final.csv', index=False)

In [164]:
feature_importance = pd.DataFrame({'name':feature_names, 'importance':model.feature_importances_},columns = ['name','importance'])
sort_importance = feature_importance.sort_values(by= ['importance'],ascending = False)
sort_importance.head(10)

,name,importance
50991,发生,0.004874
138622,民警,0.004323
197312,警方,0.004238
200352,调查,0.004145
50835,发展,0.003709
157432,现场,0.003652
144328,活动,0.003380
92445,开展,0.003379
50983,发现,0.003309
79165,实现,0.003186


In [145]:
sort_importance.to_csv(path+"sort_importance.csv",index=False)

# 使用bert进行情感分析

In [8]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import os
import codecs
import json
import random
import logging
import argparse
from tqdm import tqdm, trange

from sklearn import metrics
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_json(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        dicts = []
        with codecs.open(input_file, 'r', 'utf-8') as infs:
            for inf in infs:
                inf = inf.strip()
                dicts.append(json.loads(inf))
        return dicts


class MyPro(DataProcessor):
    '''自定义数据读取方法，针对json文件
    
    Returns:
        examples: 数据集，包含index、中文文本、类别三个部分
    '''
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_json(os.path.join(data_dir, "train.json")), 'train')

    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_json(os.path.join(data_dir, "val.json")), 'dev')

    def get_test_examples(self, data_dir):
        return self._create_examples(
            self._read_json(os.path.join(data_dir, "test.json")), 'test')

    def get_labels(self):
        return [0, 1]

    def _create_examples(self, dicts, set_type):
        examples = []
        for (i, infor) in enumerate(dicts):
            guid = "%s-%s" % (set_type, i)
            text_a = infor['question']
            label = infor['label']
            examples.append(
                InputExample(guid=guid, text_a=text_a, label=label))
        return examples


def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, show_exp=True):
    '''Loads a data file into a list of `InputBatch`s.
    Args:
        examples      : [List] 输入样本，包括question, label, index
        label_list    : [List] 所有可能的类别，可以是int、str等，如['book', 'city', ...]
        max_seq_length: [int] 文本最大长度
        tokenizer     : [Method] 分词方法
    Returns:
        features:
            input_ids  : [ListOf] token的id，在chinese模式中就是每个分词的id，对应一个word vector
            input_mask : [ListOfInt] 真实字符对应1，补全字符对应0
            segment_ids: [ListOfInt] 句子标识符，第一句全为0，第二句全为1
            label_id   : [ListOfInt] 将Label_list转化为相应的id表示
    '''
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label_map[example.label]
        if ex_index < 5 and show_exp:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

	
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)


def copy_optimizer_params_to_model(named_params_model, named_params_optimizer):
    """ Utility function for optimize_on_cpu and 16-bits training.
        Copy the parameters optimized on CPU/RAM back to the model on GPU
    """
    for (name_opti, param_opti), (name_model, param_model) in zip(named_params_optimizer, named_params_model):
        if name_opti != name_model:
            logger.error("name_opti != name_model: {} {}".format(name_opti, name_model))
            raise ValueError
        param_model.data.copy_(param_opti.data)

	
def set_optimizer_params_grad(named_params_optimizer, named_params_model, test_nan=False):
    """ Utility function for optimize_on_cpu and 16-bits training.
        Copy the gradient of the GPU parameters to the CPU/RAMM copy of the model
    """
    is_nan = False
    for (name_opti, param_opti), (name_model, param_model) in zip(named_params_optimizer, named_params_model):
        if name_opti != name_model:
            logger.error("name_opti != name_model: {} {}".format(name_opti, name_model))
            raise ValueError
        if param_model.grad is not None:
            if test_nan and torch.isnan(param_model.grad).sum() > 0:
                is_nan = True
            if param_opti.grad is None:
                param_opti.grad = torch.nn.Parameter(param_opti.data.new().resize_(*param_opti.data.size()))
            param_opti.grad.data.copy_(param_model.grad.data)
        else:
            param_opti.grad = None
    return is_nan


def val(model, processor, args, label_list, tokenizer, device):
    '''模型验证
    
    Args:
        model: 模型
	processor: 数据读取方法
	args: 参数表
	label_list: 所有可能类别
	tokenizer: 分词方法
	device
	
    Returns:
        f1: F1值
    '''
    eval_examples = processor.get_dev_examples(args.data_dir)
    eval_features = convert_examples_to_features(
        eval_examples, label_list, args.max_seq_length, tokenizer, show_exp=False)
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)
    
    model.eval()
    predict = np.zeros((0,), dtype=np.int32)
    gt = np.zeros((0,), dtype=np.int32)
    for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
 
        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)         
            pred = logits.max(1)[1]
            predict = np.hstack((predict, pred.cpu().numpy()))
            gt = np.hstack((gt, label_ids.cpu().numpy()))

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()

    print(len(gt))
    f1 = np.mean(metrics.f1_score(predict, gt, average=None))
    print(f1)

    return f1


def test(model, processor, args, label_list, tokenizer, device):
    '''模型测试
    
    Args:
        model: 模型
	processor: 数据读取方法
	args: 参数表
	label_list: 所有可能类别
	tokenizer: 分词方法
	device
	
    Returns:
        f1: F1值
    '''
    test_examples = processor.get_test_examples(args.data_dir)
    test_features = convert_examples_to_features(
        test_examples, label_list, args.max_seq_length, tokenizer)
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.long)
    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.eval_batch_size)
    
    model.eval()
    predict = np.zeros((0,), dtype=np.int32)
    gt = np.zeros((0,), dtype=np.int32)
    for input_ids, input_mask, segment_ids, label_ids in test_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
 
        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)         
            pred = logits.max(1)[1]
            predict = np.hstack((predict, pred.cpu().numpy()))
            gt = np.hstack((gt, label_ids.cpu().numpy()))

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()

    f1 = np.mean(metrics.f1_score(predict, gt, average=None))
    print('F1 score in text set is {}'.format(f1))

    return f1


def main():
    # ArgumentParser对象保存了所有必要的信息，用以将命令行参数解析为相应的python数据类型
    parser = argparse.ArgumentParser()

    # required parameters
    # 调用add_argument()向ArgumentParser对象添加命令行参数信息，这些信息告诉ArgumentParser对象如何处理命令行参数
    parser.add_argument("--data_dir",
                        default = '/data/users/zfsun3/text_classification/data/',
                        type = str,
                        #required = True,
                        help = "The input data dir. Should contain the .tsv files (or other data files) for the task.")
    parser.add_argument("--bert_model",
                        default = 'bert-base-chinese',
                        type = str,
                        #required = True,
                        help = "choose [bert-base-chinese] mode.")
    parser.add_argument("--task_name",
                        default = 'MyPro',
                        type = str,
                        #required = True,
                        help = "The name of the task to train.")
    parser.add_argument("--output_dir",
                        default = 'checkpoints/',
                        type = str,
                        #required = True,
                        help = "The output directory where the model checkpoints will be written")
    parser.add_argument("--model_save_pth",
                        default = 'checkpoints/bert_classification.pth',
                        type = str,
                        #required = True,
                        help = "The output directory where the model checkpoints will be written")

    # other parameters
    parser.add_argument("--max_seq_length",
                        default = 22,
                        type = int,
                        help = "字符串最大长度")
    parser.add_argument("--do_train",
                        default = True,
                        action = 'store_true',
                        help = "训练模式")
    parser.add_argument("--do_eval",
                        default = True,
                        action = 'store_true',
                        help = "验证模式")
    parser.add_argument("--do_lower_case",
                        default = False,
                        action = 'store_true',
                        help = "英文字符的大小写转换，对于中文来说没啥用")
    parser.add_argument("--train_batch_size",
                        default = 128,
                        type = int,
                        help = "训练时batch大小")
    parser.add_argument("--eval_batch_size",
                        default = 1,
                        type = int,
                        help = "验证时batch大小")
    parser.add_argument("--learning_rate",
                        default = 5e-5,
                        type = float,
                        help = "Adam初始学习步长")
    parser.add_argument("--num_train_epochs",
                        default = 10.0,
                        type = float,
                        help = "训练的epochs次数")
    parser.add_argument("--warmup_proportion",
                        default = 0.1,
                        type = float,
                        help = "Proportion of training to perform linear learning rate warmup for."
                               "E.g., 0.1 = 10%% of training.")
    parser.add_argument("--no_cuda",
                        default = False,
                        action = 'store_true',
                        help = "用不用CUDA")
    parser.add_argument("--local_rank",
                        default = -1,
                        type = int,
                        help = "local_rank for distributed training on gpus.")
    parser.add_argument("--seed",
                        default = 777,
                        type = int,
                        help = "初始化时的随机数种子")
    parser.add_argument("--gradient_accumulation_steps",
                        default = 1,
                        type = int,
                        help = "Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--optimize_on_cpu",
                        default = False,
                        action = 'store_true',
                        help = "Whether to perform optimization and keep the optimizer averages on CPU.")
    parser.add_argument("--fp16",
                        default = False,
                        action = 'store_true',
                        help = "Whether to use 16-bit float precision instead of 32-bit.")
    parser.add_argument("--loss_scale",
                        default = 128,
                        type = float,
                        help = "Loss scaling, positive power of 2 values can improve fp16 convergence.")

    args = parser.parse_args()

    # 对模型输入进行处理的processor，git上可能都是针对英文的processor
    processors = {'mypro': MyPro}

    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        device = torch.device("cuda", args.local_rank)
        n_gpu = 1
        torch.distributed.init_process_group(backend='nccl')
        if args.fp16:
            logger.info("16-bits training currently not supported in distributed training") 
            args.fp16 = False # (see https://github.com/pytorch/pytorch/pull/13496)
    logger.info("device %s n_gpu %d distributed training %r", device, n_gpu, bool(args.local_rank != -1))

    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            args.gradient_accumulation_steps))
    
    args.train_batch_size = int(args.train_batch_size / args.gradient_accumulation_steps)

    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

    if not args.do_train and not args.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")

    if os.path.exists(args.output_dir) and os.listdir(args.output_dir):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
    os.makedirs(args.output_dir, exist_ok=True)

    task_name = args.task_name.lower()

    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
    
    processor = processors[task_name]()
    label_list = processor.get_labels()

    tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)

    train_examples = None
    num_train_steps = None
    if args.do_train:
        train_examples = processor.get_train_examples(args.data_dir)
        num_train_steps = int(
            len(train_examples) / args.train_batch_size / args.gradient_accumulation_steps * args.num_train_epochs)

    # Prepare model
    model = BertForSequenceClassification.from_pretrained(args.bert_model, 
                cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(args.local_rank))

    if args.fp16:
    	model.half()
    model.to(device)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(model, 
                                                          device_ids=[args.local_rank],
                                                          output_device=args.local_rank)
    elif n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Prepare optimizer
    if args.fp16:
        param_optimizer = [(n, param.clone().detach().to('cpu').float().requires_grad_()) \
                            for n, param in model.named_parameters()]
    elif args.optimize_on_cpu:
        param_optimizer = [(n, param.clone().detach().to('cpu').requires_grad_()) \
                            for n, param in model.named_parameters()]
    else:
    	param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
        ]
    t_total = num_train_steps
    if args.local_rank != -1:
        t_total = t_total // torch.distributed.get_world_size()
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=args.learning_rate,
                         warmup=args.warmup_proportion,
                         t_total=t_total)

    global_step = 0
    if args.do_train:
        train_features = convert_examples_to_features(
            train_examples, label_list, args.max_seq_length, tokenizer, show_exp=False)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", args.train_batch_size)
        logger.info("  Num steps = %d", num_train_steps)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        if args.local_rank == -1:
            train_sampler = RandomSampler(train_data)
        else:
            train_sampler = DistributedSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

        model.train()
        best_score = 0
        flags = 0
        for _ in trange(int(args.num_train_epochs), desc="Epoch"):
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch
                loss = model(input_ids, segment_ids, input_mask, label_ids)
                if n_gpu > 1:
                    loss = loss.mean() # mean() to average on multi-gpu.
                if args.fp16 and args.loss_scale != 1.0:
                    # rescale loss for fp16 training
                    # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
                    loss = loss * args.loss_scale
                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps
                loss.backward()
                
                if (step + 1) % args.gradient_accumulation_steps == 0:
                    if args.fp16 or args.optimize_on_cpu:
                        if args.fp16 and args.loss_scale != 1.0:
                            # scale down gradients for fp16 training
                            for param in model.parameters():
                                if param.grad is not None:
                                    param.grad.data = param.grad.data / args.loss_scale
                        is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
                        if is_nan:
                            logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
                            args.loss_scale = args.loss_scale / 2
                            model.zero_grad()
                            continue
                        optimizer.step()
                        copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
                    else:
                        optimizer.step()
                    model.zero_grad()
    
            f1 = val(model, processor, args, label_list, tokenizer, device)
            if f1 > best_score:
                best_score = f1
                print('*f1 score = {}'.format(f1))
                flags = 0
                checkpoint = {
                    'state_dict': model.state_dict()
                }
                torch.save(checkpoint, args.model_save_pth)
            else:
                print('f1 score = {}'.format(f1))
                flags += 1
                if flags >=6:
                    break

    model.load_state_dict(torch.load(args.model_save_pth)['state_dict'])
    test(model, processor, args, label_list, tokenizer, device)
	

if __name__ == '__main__':
	main()

usage: ipykernel_launcher.py [-h] [--data_dir DATA_DIR]
                             [--bert_model BERT_MODEL] [--task_name TASK_NAME]
                             [--output_dir OUTPUT_DIR]
                             [--model_save_pth MODEL_SAVE_PTH]
                             [--max_seq_length MAX_SEQ_LENGTH] [--do_train]
                             [--do_eval] [--do_lower_case]
                             [--train_batch_size TRAIN_BATCH_SIZE]
                             [--eval_batch_size EVAL_BATCH_SIZE]
                             [--learning_rate LEARNING_RATE]
                             [--num_train_epochs NUM_TRAIN_EPOCHS]
                             [--warmup_proportion WARMUP_PROPORTION]
                             [--no_cuda] [--local_rank LOCAL_RANK]
                             [--seed SEED]
                             [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                             [--optimize_on_cpu] [--fp16]
                          

SystemExit: 2

In [9]:
! pip install pytorch-pretrained-bert

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
